In [ ]:
%reset -f
%matplotlib inline
from tjmonopix.analysis.plotting import plotting
plot=plotting()

start_dif=0
VHLrange=40
max_occ = 500
DUtoe=20.18
threshold=350
sigma=14
folder='IC_PMOS_PW3_PS6'

plot.plot_scurve_histogram(max_occ+10, DUtoe, folder, 50, 16, 6)

In [ ]:
plot.fit_scurvedata(VHLrange, start_dif, max_occ, DUtoe, threshold, sigma, folder)

In [ ]:
plot.plot_thr_dispersion(folder)
plot.plot_enc_dispersion(folder)

In [ ]:
# %reset -f
# from tjmonopix.analysis.plotting import plotting
# plot=plotting()
# folder='HV_PW3_PS6_HV1_6'
plot.dplot(folder, 'threshold')
plot.dplot(folder, 'enc')

In [ ]:
import numpy as np
enctop = np.load('./'+folder+'/enctop.npy')
encbot = np.load('./'+folder+'/enctop.npy')
#ee=np.where((enc > 1)&(enc < 3))[0]
maskedbot=np.where(encbot==0)[0]
maskedtop=np.where(encbot==0)[0]
#print ee
masked = len(maskedbot)+len(maskedtop)
print masked

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf

xhits = range(start_dif,VHLrange+start_dif+1)

scurves = np.load('./'+folder+'/scurvedatatop.npy')
scurves[:,0]=0
scurves[:,1]=0
pix=11063
print scurves[pix,:]
popt, _ = curve_fit(plot.scurve, xhits, scurves[pix,:], p0=[max_occ, threshold, sigma], check_finite=False)
print popt
pt=plt.plot(scurves[pix,:], 'ro')

In [ ]:
index=3829
col = index // 112
row = index-col*112
row2 = (index-col*112)+112
print col
print row
print row2